In [ ]:
import pandas as pd
import numpy as np
from sklearn import metrics
import tensorflow as tf

# 1. MNIST_784

In [ ]:
from keras.datasets.mnist import load_data

(X_train, y_train), (X_test, y_test) = load_data()
print(X_test.shape, y_test.shape)

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
np.random.seed(0)

class_names = ['0', '1', '2', '3', '4', '5', '6', '7','8','9']
sample_size = 9
random_idx = np.random.randint(10000, size=sample_size)

plt.figure(figsize=(5,5))
for i, idx in enumerate(random_idx):
    plt.subplot(3,3,i+1)
    plt.xticks([])
    plt.yticks([])
    plt.imshow(X_test[idx], cmap='gray')
    plt.xlabel(class_names[y_test[idx]])
plt.show()

In [ ]:
X_test = np.repeat(X_test[..., np.newaxis], 3, -1)
X_test.shape

In [ ]:
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np

class_names = ['0', '1', '2', '3', '4', '5', '6', '7','8','9']
y_list = []

for i in range(100):
    if y_test[i] in y_list:
        continue
    y_list.append(y_test[i])    
    some_digit = X_test[i]
    # some_digit_image = some_digit.reshape(28, 28)

    plt.imshow(some_digit, cmap = "binary")
    plt.axis("off")
    plt.show()
    print(y_test[i], class_names[y_test[i]])

In [ ]:
pd.Series(y_test).value_counts()

In [ ]:
idx_7 = []
idx_9 = []
for i in range(len(y_test)):
    if list(y_test)[i] == 7:
        idx_7.append(i)
    if list(y_test)[i] == 9:
        idx_9.append(i)
print(len(idx_7), len(idx_9))

In [ ]:
idx = idx_7[:50] + idx_9[:50]
idx.sort()

In [ ]:
X = []
y = []
for i in idx:
    X.append(X_test[i])
    y.append(y_test[i])
X = np.array(X)
y = np.array(y)
print(X.shape, y.shape)

In [ ]:
pd.Series(y).value_counts()

In [ ]:
y_hard = [0.00 if x==7 else x for x in y]
y_hard = [1.00 if x==9 else x for x in y_hard]
pd.Series(y_hard).value_counts()

In [ ]:
from sklearn.model_selection import StratifiedKFold
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state = 2)

epochs = 10
batch = 32

from keras.callbacks import EarlyStopping, ModelCheckpoint

In [ ]:
# ResNet
import tensorflow as tf
import numpy as np
from keras import optimizers

from tensorflow.keras.applications.resnet50 import preprocess_input, decode_predictions
from tensorflow.keras.applications.resnet50 import ResNet50

from tensorflow.keras.preprocessing import image

from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Conv2D, MaxPooling2D, Flatten
from tensorflow.keras.models import Model, Sequential

def create_model():
    # Remove "Fully Connected layer"
    resnet = ResNet50(weights='imagenet', include_top=False)
    # Parameter update
    for layer in resnet.layers:
        layer.trainable = True
    # Add "Fully connected layer" for binary classification
    r = resnet.output
    r = GlobalAveragePooling2D()(r)
    r = Dense(16, activation='relu')(r)
    predictions = Dense(1, activation='sigmoid')(r)
    model = Model(resnet.input, predictions)
    return model

In [ ]:
X

In [ ]:
y_hard = pd.DataFrame(y_hard)
y_hard

# 1-0. Generating Prob_labels

In [ ]:
gen_model = create_model()   
gen_model.compile(loss='BinaryCrossentropy', optimizer=optimizers.Adam(learning_rate = 0.0005), metrics=['accuracy'])
history = gen_model.fit(X, y_hard, validation_split=0.2, epochs=epochs, verbose=0, batch_size=batch)#, callbacks=[early_stopping])
plt.plot(history.history['loss'], label='loss')
plt.ylim([0, 1])
plt.xlabel('Iteration',fontweight="bold",fontsize = 15)
plt.ylabel('Loss',fontweight="bold",fontsize = 15)
plt.title("Cost Function",fontweight="bold",fontsize = 20)
plt.legend()
plt.show()
result = gen_model.predict(X, verbose=0)
prob_label = list(result.reshape(len(X),))
y = pd.DataFrame(prob_label)
y

In [ ]:
#
c = 0.5
y_edge = []
for i in range(len(y)):
    if list(y_hard[0])[i] == 0:
        if list(y[0])[i] <= c:
            y_edge.append(0)  # easy sample
        else:
            y_edge.append(2) # hard sample
    if list(y_hard[0])[i] == 1:
        if list(y[0])[i] >= 1-c:
            y_edge.append(0)  # easy sample
        else:
            y_edge.append(2) # hard sample
y_edge = pd.DataFrame(y_edge)
y_edge.value_counts()   

In [ ]:
#
edge_list = list(y_edge[y_edge[0] == 2].index)
normal_list = list(y_edge[y_edge[0] == 0].index)
print(edge_list)

In [ ]:
r = list(y_edge[0]).count(0)/list(y_edge[0]).count(2)   # normal/edge
alpha = (r-1)/(2*r)
print(r, alpha)

# 1-1. Focal(Hard) and SLS(Hard/alpha)

In [ ]:
#
r = list(y_edge[0]).count(0)/list(y_edge[0]).count(2)   # normal/edge
alpha = (r-1)/(2*r)
B = [0.00, alpha]

for t in range(10):    # 10 times repeat      
    res = pd.DataFrame({'Focal':[0, 0, 0]}, index = ['Total','Edge','Normal']) 
    # Focal
    print('#'*50,'Focal','#'*50)
    list_total = []
    list_edge = []
    list_normal = []  
    focal_model = create_model()   

    n_iter = 0
    for train_index, test_index in skf.split(X, y_edge):  # straticiation by y_edge
        n_iter += 1
        X_train = X[train_index]
        y_train= y_hard.iloc[train_index]     # train with hard labels
        if n_iter == 1:
            print(y_train.value_counts())
        X_test = X[test_index]
        y_test= y_hard.iloc[test_index]     # test with hard labels
        test_edge_list = []
        for index in edge_list:
            if index in test_index:
                test_edge_list.append(index)
        X_test_edge = X[test_edge_list]
        y_test_edge = y_hard.iloc[test_edge_list]     # test with hard labels
        test_normal_list = []
        for index in normal_list:
            if index in test_index:
                test_normal_list.append(index)
        X_test_normal = X[test_normal_list]
        y_test_normal = y_hard.iloc[test_normal_list]     # test with hard labels

        X_train = np.array(X_train)
        y_train = np.array(y_train)
        y_train = y_train.astype(float)    
        X_test = np.array(X_test)
        y_test = np.array(y_test)
        y_test = y_test.astype(float)
        X_test_edge = np.array(X_test_edge)
        y_test_edge = np.array(y_test_edge)
        y_test_edge = y_test_edge.astype(float)
        X_test_normal = np.array(X_test_normal)
        y_test_normal = np.array(y_test_normal)
        y_test_normal = y_test_normal.astype(float)


        focal_model.compile(loss='BinaryFocalCrossentropy', optimizer=optimizers.Adam(learning_rate = 0.0005), metrics=['accuracy'])
        history = focal_model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=epochs, verbose=0, batch_size=batch)#, callbacks=[early_stopping])
#         plt.plot(history.history['loss'], label='loss')
#         plt.ylim([0, 1])
#         plt.xlabel('Iteration',fontweight="bold",fontsize = 15)
#         plt.ylabel('Loss',fontweight="bold",fontsize = 15)
#         plt.title("Cost Function",fontweight="bold",fontsize = 20)
#         plt.legend()
#         plt.show()

        # TEST (total)
        predicted_total = np.round(focal_model.predict(X_test, verbose=0))
        list_total.append(metrics.accuracy_score(y_test, predicted_total))
        # TEST (edge)
        predicted_edge = np.round(focal_model.predict(X_test_edge, verbose=0))
        list_edge.append(metrics.accuracy_score(y_test_edge, predicted_edge))
        # TEST (normal)
        predicted_normal = np.round(focal_model.predict(X_test_normal, verbose=0))
        list_normal.append(metrics.accuracy_score(y_test_normal, predicted_normal))
            
    res['Focal'] = [np.mean(list_total), np.mean(list_edge), np.mean(list_normal)]
    print([np.mean(list_total), np.mean(list_edge), np.mean(list_normal)])
    
    for b in B:
        print('#'*50,'SLS',b,'#'*50)
        y_sls = []
        for i in range(len(y_hard)):
            if list(y_hard[0])[i] == 0:
                if prob_label[i] <= c:
                    y_sls.append(b)  # easy sample
                else:
                    y_sls.append(0) # hard sample
            if list(y_hard[0])[i] == 1:
                if prob_label[i] >= 1-c:
                    y_sls.append(1-b)  # easy sample
                else:
                    y_sls.append(1) # hard sample
        y_sls = pd.DataFrame(y_sls)       

        sls_total = []
        sls_edge = []
        sls_normal = []
        model_sls = create_model()

        n_iter = 0
        for train_index, test_index in skf.split(X, y_edge):  # straticiation by y_edge
            n_iter += 1
            X_train = X[train_index]
            y_sls_train = y_sls.iloc[train_index]     # train with sls labels
            if n_iter == 1:
                print(y_sls_train.value_counts())
            X_test = X[test_index]
            y_test= y_hard.iloc[test_index]     # test with hard labels
            test_edge_list = []
            for index in edge_list:
                if index in test_index:
                    test_edge_list.append(index)
            X_test_edge = X[test_edge_list]
            y_test_edge = y_hard.iloc[test_edge_list]     # test with hard labels
            test_normal_list = []
            for index in normal_list:
                if index in test_index:
                    test_normal_list.append(index)
            X_test_normal = X[test_normal_list]
            y_test_normal = y_hard.iloc[test_normal_list]     # test with hard labels

            X_train = np.array(X_train)
            y_sls_train = np.array(y_sls_train)
            y_sls_train = y_sls_train.astype(float)
            X_test = np.array(X_test)
            y_test = np.array(y_test)
            y_test = y_test.astype(float)
            X_test_edge = np.array(X_test_edge)
            y_test_edge = np.array(y_test_edge)
            y_test_edge = y_test_edge.astype(float)
            X_test_normal = np.array(X_test_normal)
            y_test_normal = np.array(y_test_normal)
            y_test_normal = y_test_normal.astype(float)

            # MLP_BCE(y_005)
            model_sls.compile(loss='BinaryCrossentropy', optimizer=optimizers.Adam(learning_rate = 0.0005), metrics=['accuracy'])
            history = model_sls.fit(X_train, y_sls_train, validation_data=(X_test, y_test), epochs=epochs, verbose=0, batch_size=batch)#, callbacks=[early_stopping])
#             plt.plot(history.history['loss'], label='loss')
#             plt.ylim([0, 1])
#             plt.xlabel('Iteration',fontweight="bold",fontsize = 15)
#             plt.ylabel('Loss',fontweight="bold",fontsize = 15)
#             plt.title("Cost Function",fontweight="bold",fontsize = 20)
#             plt.legend()
#             plt.show()
            
            # TEST (total)
            predicted_total = np.round(model_sls.predict(X_test, verbose=0))
            sls_total.append(metrics.accuracy_score(y_test, predicted_total))
            # TEST (edge)
            predicted_edge = np.round(model_sls.predict(X_test_edge, verbose=0))
            sls_edge.append(metrics.accuracy_score(y_test_edge, predicted_edge))
            # TEST (normal)
            predicted_normal = np.round(model_sls.predict(X_test_normal, verbose=0))
            sls_normal.append(metrics.accuracy_score(y_test_normal, predicted_normal))
                       
        res['SLS({})'.format(b)] = [np.mean(sls_total), np.mean(sls_edge), np.mean(sls_normal)]
        print([np.mean(sls_total), np.mean(sls_edge), np.mean(sls_normal)])       
    res.to_csv("RES_MNIST_5CV(SLS_c0.5)_alphaimproved.csv", mode = 'a', float_format='%.4g')

# 2. Fashion_MNIST

In [ ]:
from keras.datasets.fashion_mnist import load_data

(X_train, y_train), (X_test, y_test) = load_data()
print(X_test.shape, y_test.shape)

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
np.random.seed(0)

class_names = ['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat', 'Sandal', 'Shirt', 'Sneaker','Bag','Ankle boot']
sample_size = 9
random_idx = np.random.randint(10000, size=sample_size)

plt.figure(figsize=(5,5))
for i, idx in enumerate(random_idx):
    plt.subplot(3,3,i+1)
    plt.xticks([])
    plt.yticks([])
    plt.imshow(X_test[idx], cmap='gray')
    plt.xlabel(class_names[y_test[idx]])
plt.show()

In [ ]:
X_test = np.repeat(X_test[..., np.newaxis], 3, -1)
X_test.shape

In [ ]:
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np

class_names = ['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat', 'Sandal', 'Shirt', 'Sneaker','Bag','Ankle boot']
y_list = []

for i in range(100):
    if y_test[i] in y_list:
        continue
    y_list.append(y_test[i])    
    some_digit = X_test[i]
    # some_digit_image = some_digit.reshape(28, 28)

    plt.imshow(some_digit, cmap = "binary")
    plt.axis("off")
    plt.show()
    print(y_test[i], class_names[y_test[i]])

In [ ]:
pd.Series(y_test).value_counts()

In [ ]:
# Picking only label (4, 6)->0 & 2->1
idx_2 = []
idx_4 = []
# idx_6 = []
for i in range(len(y_test)):
    if list(y_test)[i] == 2:
        idx_2.append(i)
    if list(y_test)[i] == 4:
        idx_4.append(i)
#     if list(y_test)[i] == 6:
#         idx_6.append(i)
print(len(idx_2), len(idx_4)) #, len(idx_6))

In [ ]:
idx = idx_2[:50] + idx_4[:50] # + idx_6[:110]
idx.sort()

In [ ]:
X = []
y = []
for i in idx:
    X.append(X_test[i])
    y.append(y_test[i])
X = np.array(X)
y = np.array(y)
print(X.shape, y.shape)

In [ ]:
pd.Series(y).value_counts()

In [ ]:
y_hard = [0.00 if x==2 else x for x in y]
y_hard = [1.00 if x==4 else x for x in y_hard]
pd.Series(y_hard).value_counts()

In [ ]:
from sklearn.model_selection import StratifiedKFold
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state = 2)

epochs = 10
batch = 32

from keras.callbacks import EarlyStopping, ModelCheckpoint

In [ ]:
# ResNet
import tensorflow as tf
import numpy as np
from keras import optimizers

from tensorflow.keras.applications.resnet50 import preprocess_input, decode_predictions
from tensorflow.keras.applications.resnet50 import ResNet50

from tensorflow.keras.preprocessing import image

from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Conv2D, MaxPooling2D, Flatten
from tensorflow.keras.models import Model, Sequential

def create_model():
    # Remove "Fully Connected layer"
    resnet = ResNet50(weights='imagenet', include_top=False)
    # Parameter update
    for layer in resnet.layers:
        layer.trainable = True
    # Add "Fully connected layer" for binary classification
    r = resnet.output
    r = GlobalAveragePooling2D()(r)
    r = Dense(16, activation='relu')(r)
    predictions = Dense(1, activation='sigmoid')(r)
    model = Model(resnet.input, predictions)
    return model

In [ ]:
X

In [ ]:
y_hard = pd.DataFrame(y_hard)
y_hard

# 2-0. Generating Prob_labels

In [ ]:
gen_model = create_model()   
gen_model.compile(loss='BinaryCrossentropy', optimizer=optimizers.Adam(learning_rate = 0.0003), metrics=['accuracy'])
history = gen_model.fit(X, y_hard, validation_split=0.2, epochs=epochs, verbose=0, batch_size=batch)#, callbacks=[early_stopping])
plt.plot(history.history['loss'], label='loss')
plt.ylim([0, 1])
plt.xlabel('Iteration',fontweight="bold",fontsize = 15)
plt.ylabel('Loss',fontweight="bold",fontsize = 15)
plt.title("Cost Function",fontweight="bold",fontsize = 20)
plt.legend()
plt.show()
result = gen_model.predict(X, verbose=0)
prob_label = list(result.reshape(len(X),))
y = pd.DataFrame(prob_label)
y

In [ ]:
#
c = 0.5
y_edge = []
for i in range(len(y)):
    if list(y_hard[0])[i] == 0:
        if list(y[0])[i] <= c:
            y_edge.append(0)  # easy sample
        else:
            y_edge.append(2) # hard sample
    if list(y_hard[0])[i] == 1:
        if list(y[0])[i] >= 1-c:
            y_edge.append(0)  # easy sample
        else:
            y_edge.append(2) # hard sample
y_edge = pd.DataFrame(y_edge)
y_edge.value_counts()   

In [ ]:
#
edge_list = list(y_edge[y_edge[0] == 2].index)
normal_list = list(y_edge[y_edge[0] == 0].index)
print(edge_list)

In [ ]:
r = list(y_edge[0]).count(0)/list(y_edge[0]).count(2)   # normal/edge
alpha = (r-1)/(2*r)
print(r, alpha)

# 2-1. Focal(Hard) and SLS(Hard/alpha)

In [ ]:
#
r = list(y_edge[0]).count(0)/list(y_edge[0]).count(2)   # normal/edge
alpha = (r-1)/(2*r)
B = [0.00, alpha]

for t in range(10):    # 10 times repeat      
    res = pd.DataFrame({'Focal':[0, 0, 0]}, index = ['Total','Edge','Normal']) 
    # Focal
    print('#'*50,'Focal','#'*50)
    list_total = []
    list_edge = []
    list_normal = []  
    focal_model = create_model()   

    n_iter = 0
    for train_index, test_index in skf.split(X, y_edge):  # straticiation by y_edge
        n_iter += 1
        X_train = X[train_index]
        y_train= y_hard.iloc[train_index]     # train with hard labels
        if n_iter == 1:
            print(y_train.value_counts())
        X_test = X[test_index]
        y_test= y_hard.iloc[test_index]     # test with hard labels
        test_edge_list = []
        for index in edge_list:
            if index in test_index:
                test_edge_list.append(index)
        X_test_edge = X[test_edge_list]
        y_test_edge = y_hard.iloc[test_edge_list]     # test with hard labels
        test_normal_list = []
        for index in normal_list:
            if index in test_index:
                test_normal_list.append(index)
        X_test_normal = X[test_normal_list]
        y_test_normal = y_hard.iloc[test_normal_list]     # test with hard labels

        X_train = np.array(X_train)
        y_train = np.array(y_train)
        y_train = y_train.astype(float)    
        X_test = np.array(X_test)
        y_test = np.array(y_test)
        y_test = y_test.astype(float)
        X_test_edge = np.array(X_test_edge)
        y_test_edge = np.array(y_test_edge)
        y_test_edge = y_test_edge.astype(float)
        X_test_normal = np.array(X_test_normal)
        y_test_normal = np.array(y_test_normal)
        y_test_normal = y_test_normal.astype(float)


        focal_model.compile(loss='BinaryFocalCrossentropy', optimizer=optimizers.Adam(learning_rate = 0.0003), metrics=['accuracy'])
        history = focal_model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=epochs, verbose=0, batch_size=batch)#, callbacks=[early_stopping])
#         plt.plot(history.history['loss'], label='loss')
#         plt.ylim([0, 1])
#         plt.xlabel('Iteration',fontweight="bold",fontsize = 15)
#         plt.ylabel('Loss',fontweight="bold",fontsize = 15)
#         plt.title("Cost Function",fontweight="bold",fontsize = 20)
#         plt.legend()
#         plt.show()

        # TEST (total)
        predicted_total = np.round(focal_model.predict(X_test, verbose=0))
        list_total.append(metrics.accuracy_score(y_test, predicted_total))
        # TEST (edge)
        predicted_edge = np.round(focal_model.predict(X_test_edge, verbose=0))
        list_edge.append(metrics.accuracy_score(y_test_edge, predicted_edge))
        # TEST (normal)
        predicted_normal = np.round(focal_model.predict(X_test_normal, verbose=0))
        list_normal.append(metrics.accuracy_score(y_test_normal, predicted_normal))
            
    res['Focal'] = [np.mean(list_total), np.mean(list_edge), np.mean(list_normal)]
    print([np.mean(list_total), np.mean(list_edge), np.mean(list_normal)])
    
    for b in B:
        print('#'*50,'SLS',b,'#'*50)
        y_sls = []
        for i in range(len(y_hard)):
            if list(y_hard[0])[i] == 0:
                if prob_label[i] <= c:
                    y_sls.append(b)  # easy sample
                else:
                    y_sls.append(0) # hard sample
            if list(y_hard[0])[i] == 1:
                if prob_label[i] >= 1-c:
                    y_sls.append(1-b)  # easy sample
                else:
                    y_sls.append(1) # hard sample
        y_sls = pd.DataFrame(y_sls)       

        sls_total = []
        sls_edge = []
        sls_normal = []
        model_sls = create_model()

        n_iter = 0
        for train_index, test_index in skf.split(X, y_edge):  # straticiation by y_edge
            n_iter += 1
            X_train = X[train_index]
            y_sls_train = y_sls.iloc[train_index]     # train with sls labels
            if n_iter == 1:
                print(y_sls_train.value_counts())
            X_test = X[test_index]
            y_test= y_hard.iloc[test_index]     # test with hard labels
            test_edge_list = []
            for index in edge_list:
                if index in test_index:
                    test_edge_list.append(index)
            X_test_edge = X[test_edge_list]
            y_test_edge = y_hard.iloc[test_edge_list]     # test with hard labels
            test_normal_list = []
            for index in normal_list:
                if index in test_index:
                    test_normal_list.append(index)
            X_test_normal = X[test_normal_list]
            y_test_normal = y_hard.iloc[test_normal_list]     # test with hard labels

            X_train = np.array(X_train)
            y_sls_train = np.array(y_sls_train)
            y_sls_train = y_sls_train.astype(float)
            X_test = np.array(X_test)
            y_test = np.array(y_test)
            y_test = y_test.astype(float)
            X_test_edge = np.array(X_test_edge)
            y_test_edge = np.array(y_test_edge)
            y_test_edge = y_test_edge.astype(float)
            X_test_normal = np.array(X_test_normal)
            y_test_normal = np.array(y_test_normal)
            y_test_normal = y_test_normal.astype(float)

            # MLP_BCE(y_005)
            model_sls.compile(loss='BinaryCrossentropy', optimizer=optimizers.Adam(learning_rate = 0.0003), metrics=['accuracy'])
            history = model_sls.fit(X_train, y_sls_train, validation_data=(X_test, y_test), epochs=epochs, verbose=0, batch_size=batch)#, callbacks=[early_stopping])
#             plt.plot(history.history['loss'], label='loss')
#             plt.ylim([0, 1])
#             plt.xlabel('Iteration',fontweight="bold",fontsize = 15)
#             plt.ylabel('Loss',fontweight="bold",fontsize = 15)
#             plt.title("Cost Function",fontweight="bold",fontsize = 20)
#             plt.legend()
#             plt.show()
            
            # TEST (total)
            predicted_total = np.round(model_sls.predict(X_test, verbose=0))
            sls_total.append(metrics.accuracy_score(y_test, predicted_total))
            # TEST (edge)
            predicted_edge = np.round(model_sls.predict(X_test_edge, verbose=0))
            sls_edge.append(metrics.accuracy_score(y_test_edge, predicted_edge))
            # TEST (normal)
            predicted_normal = np.round(model_sls.predict(X_test_normal, verbose=0))
            sls_normal.append(metrics.accuracy_score(y_test_normal, predicted_normal))
                       
        res['SLS({})'.format(b)] = [np.mean(sls_total), np.mean(sls_edge), np.mean(sls_normal)]
        print([np.mean(sls_total), np.mean(sls_edge), np.mean(sls_normal)])       
    res.to_csv("RES_F_MNIST_5CV(SLS_c0.5)_alphaimproved.csv", mode = 'a', float_format='%.4g')

# 3. CIFAR-10

In [ ]:
from tensorflow.keras.datasets import cifar10

(X_train, y_train), (X_test, y_test) = cifar10.load_data()
print(X_test.shape, y_test.shape)

In [ ]:
y_test = y_test.reshape(10000,)

In [ ]:
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np

class_names = ['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse','ship','truck']
y_list = []

for i in range(100):
    if y_test[i] in y_list:
        continue
    y_list.append(y_test[i])    
    some_digit = X_test[i]
    # some_digit_image = some_digit.reshape(28, 28)

    plt.imshow(some_digit, cmap = "binary")
    plt.axis("off")
    plt.show()
    print(y_test[i], class_names[y_test[i]])

In [ ]:
pd.Series(y_test).value_counts()

In [ ]:
# Picking only label (1, 7)->0 & 9->1
idx_1 = []
# idx_7 = []
idx_9 = []
for i in range(len(y_test)):
    if list(y_test)[i] == 1:
        idx_1.append(i)
#     if list(y_test)[i] == 7:
#         idx_7.append(i)
    if list(y_test)[i] == 9:
        idx_9.append(i)
print(len(idx_1), len(idx_9))

In [ ]:
idx = idx_1[:50] + idx_9[:50]
idx.sort()

In [ ]:
X = []
y = []
for i in idx:
    X.append(X_test[i])
    y.append(y_test[i])
X = np.array(X)
y = np.array(y)
print(X.shape, y.shape)

In [ ]:
pd.Series(y).value_counts()

In [ ]:
y_hard = [0.00 if x==1 else x for x in y]
y_hard = [1.00 if x==9 else x for x in y_hard]
pd.Series(y_hard).value_counts()

In [ ]:
from sklearn.model_selection import StratifiedKFold
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state = 2)

epochs = 10
batch = 32

from keras.callbacks import EarlyStopping, ModelCheckpoint

In [ ]:
# ResNet
import tensorflow as tf
import numpy as np
from keras import optimizers

from tensorflow.keras.applications.resnet50 import preprocess_input, decode_predictions
from tensorflow.keras.applications.resnet50 import ResNet50

from tensorflow.keras.preprocessing import image

from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Conv2D, MaxPooling2D, Flatten
from tensorflow.keras.models import Model, Sequential

def create_model():
    # Remove "Fully Connected layer"
    resnet = ResNet50(weights='imagenet', include_top=False)
    # Parameter update
    for layer in resnet.layers:
        layer.trainable = True
    # Add "Fully connected layer" for binary classification
    r = resnet.output
    r = GlobalAveragePooling2D()(r)
    r = Dense(16, activation='relu')(r)
    predictions = Dense(1, activation='sigmoid')(r)
    model = Model(resnet.input, predictions)
    return model

In [ ]:
X

In [ ]:
y_hard = pd.DataFrame(y_hard)
y_hard

# 3-0. Generating Prob_labels

In [ ]:
gen_model = create_model()   
gen_model.compile(loss='BinaryCrossentropy', optimizer=optimizers.Adam(learning_rate = 0.0005), metrics=['accuracy'])
history = gen_model.fit(X, y_hard, validation_split=0.2, epochs=epochs, verbose=0, batch_size=batch)#, callbacks=[early_stopping])
plt.plot(history.history['loss'], label='loss')
plt.ylim([0, 1])
plt.xlabel('Iteration',fontweight="bold",fontsize = 15)
plt.ylabel('Loss',fontweight="bold",fontsize = 15)
plt.title("Cost Function",fontweight="bold",fontsize = 20)
plt.legend()
plt.show()
result = gen_model.predict(X, verbose=0)
prob_label = list(result.reshape(len(X),))
y = pd.DataFrame(prob_label)
y

In [ ]:
#
c = 0.5
y_edge = []
for i in range(len(y)):
    if list(y_hard[0])[i] == 0:
        if list(y[0])[i] <= c:
            y_edge.append(0)  # easy sample
        else:
            y_edge.append(2) # hard sample
    if list(y_hard[0])[i] == 1:
        if list(y[0])[i] >= 1-c:
            y_edge.append(0)  # easy sample
        else:
            y_edge.append(2) # hard sample
y_edge = pd.DataFrame(y_edge)
y_edge.value_counts()   

In [ ]:
#
edge_list = list(y_edge[y_edge[0] == 2].index)
normal_list = list(y_edge[y_edge[0] == 0].index)
print(edge_list)

In [ ]:
r = list(y_edge[0]).count(0)/list(y_edge[0]).count(2)   # normal/edge
alpha = (r-1)/(2*r)
print(r, alpha)

# 3-1. Focal(Hard) and SLS(Hard/alpha)

In [ ]:
#
r = list(y_edge[0]).count(0)/list(y_edge[0]).count(2)   # normal/edge
alpha = (r-1)/(2*r)
B = [0.00, alpha]

for t in range(10):    # 10 times repeat      
    res = pd.DataFrame({'Focal':[0, 0, 0]}, index = ['Total','Edge','Normal']) 
    # Focal
    print('#'*50,'Focal','#'*50)
    list_total = []
    list_edge = []
    list_normal = []  
    focal_model = create_model()   

    n_iter = 0
    for train_index, test_index in skf.split(X, y_edge):  # straticiation by y_edge
        n_iter += 1
        X_train = X[train_index]
        y_train= y_hard.iloc[train_index]     # train with hard labels
        if n_iter == 1:
            print(y_train.value_counts())
        X_test = X[test_index]
        y_test= y_hard.iloc[test_index]     # test with hard labels
        test_edge_list = []
        for index in edge_list:
            if index in test_index:
                test_edge_list.append(index)
        X_test_edge = X[test_edge_list]
        y_test_edge = y_hard.iloc[test_edge_list]     # test with hard labels
        test_normal_list = []
        for index in normal_list:
            if index in test_index:
                test_normal_list.append(index)
        X_test_normal = X[test_normal_list]
        y_test_normal = y_hard.iloc[test_normal_list]     # test with hard labels

        X_train = np.array(X_train)
        y_train = np.array(y_train)
        y_train = y_train.astype(float)    
        X_test = np.array(X_test)
        y_test = np.array(y_test)
        y_test = y_test.astype(float)
        X_test_edge = np.array(X_test_edge)
        y_test_edge = np.array(y_test_edge)
        y_test_edge = y_test_edge.astype(float)
        X_test_normal = np.array(X_test_normal)
        y_test_normal = np.array(y_test_normal)
        y_test_normal = y_test_normal.astype(float)


        focal_model.compile(loss='BinaryFocalCrossentropy', optimizer=optimizers.Adam(learning_rate = 0.0005), metrics=['accuracy'])
        history = focal_model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=epochs, verbose=0, batch_size=batch)#, callbacks=[early_stopping])
#         plt.plot(history.history['loss'], label='loss')
#         plt.ylim([0, 1])
#         plt.xlabel('Iteration',fontweight="bold",fontsize = 15)
#         plt.ylabel('Loss',fontweight="bold",fontsize = 15)
#         plt.title("Cost Function",fontweight="bold",fontsize = 20)
#         plt.legend()
#         plt.show()

        # TEST (total)
        predicted_total = np.round(focal_model.predict(X_test, verbose=0))
        list_total.append(metrics.accuracy_score(y_test, predicted_total))
        # TEST (edge)
        predicted_edge = np.round(focal_model.predict(X_test_edge, verbose=0))
        list_edge.append(metrics.accuracy_score(y_test_edge, predicted_edge))
        # TEST (normal)
        predicted_normal = np.round(focal_model.predict(X_test_normal, verbose=0))
        list_normal.append(metrics.accuracy_score(y_test_normal, predicted_normal))
            
    res['Focal'] = [np.mean(list_total), np.mean(list_edge), np.mean(list_normal)]
    print([np.mean(list_total), np.mean(list_edge), np.mean(list_normal)])
    
    for b in B:
        print('#'*50,'SLS',b,'#'*50)
        y_sls = []
        for i in range(len(y_hard)):
            if list(y_hard[0])[i] == 0:
                if prob_label[i] <= c:
                    y_sls.append(b)  # easy sample
                else:
                    y_sls.append(0) # hard sample
            if list(y_hard[0])[i] == 1:
                if prob_label[i] >= 1-c:
                    y_sls.append(1-b)  # easy sample
                else:
                    y_sls.append(1) # hard sample
        y_sls = pd.DataFrame(y_sls)       

        sls_total = []
        sls_edge = []
        sls_normal = []
        model_sls = create_model()

        n_iter = 0
        for train_index, test_index in skf.split(X, y_edge):  # straticiation by y_edge
            n_iter += 1
            X_train = X[train_index]
            y_sls_train = y_sls.iloc[train_index]     # train with sls labels
            if n_iter == 1:
                print(y_sls_train.value_counts())
            X_test = X[test_index]
            y_test= y_hard.iloc[test_index]     # test with hard labels
            test_edge_list = []
            for index in edge_list:
                if index in test_index:
                    test_edge_list.append(index)
            X_test_edge = X[test_edge_list]
            y_test_edge = y_hard.iloc[test_edge_list]     # test with hard labels
            test_normal_list = []
            for index in normal_list:
                if index in test_index:
                    test_normal_list.append(index)
            X_test_normal = X[test_normal_list]
            y_test_normal = y_hard.iloc[test_normal_list]     # test with hard labels

            X_train = np.array(X_train)
            y_sls_train = np.array(y_sls_train)
            y_sls_train = y_sls_train.astype(float)
            X_test = np.array(X_test)
            y_test = np.array(y_test)
            y_test = y_test.astype(float)
            X_test_edge = np.array(X_test_edge)
            y_test_edge = np.array(y_test_edge)
            y_test_edge = y_test_edge.astype(float)
            X_test_normal = np.array(X_test_normal)
            y_test_normal = np.array(y_test_normal)
            y_test_normal = y_test_normal.astype(float)

            # MLP_BCE(y_005)
            model_sls.compile(loss='BinaryCrossentropy', optimizer=optimizers.Adam(learning_rate = 0.0005), metrics=['accuracy'])
            history = model_sls.fit(X_train, y_sls_train, validation_data=(X_test, y_test), epochs=epochs, verbose=0, batch_size=batch)#, callbacks=[early_stopping])
#             plt.plot(history.history['loss'], label='loss')
#             plt.ylim([0, 1])
#             plt.xlabel('Iteration',fontweight="bold",fontsize = 15)
#             plt.ylabel('Loss',fontweight="bold",fontsize = 15)
#             plt.title("Cost Function",fontweight="bold",fontsize = 20)
#             plt.legend()
#             plt.show()
            
            # TEST (total)
            predicted_total = np.round(model_sls.predict(X_test, verbose=0))
            sls_total.append(metrics.accuracy_score(y_test, predicted_total))
            # TEST (edge)
            predicted_edge = np.round(model_sls.predict(X_test_edge, verbose=0))
            sls_edge.append(metrics.accuracy_score(y_test_edge, predicted_edge))
            # TEST (normal)
            predicted_normal = np.round(model_sls.predict(X_test_normal, verbose=0))
            sls_normal.append(metrics.accuracy_score(y_test_normal, predicted_normal))
                       
        res['SLS({})'.format(b)] = [np.mean(sls_total), np.mean(sls_edge), np.mean(sls_normal)]
        print([np.mean(sls_total), np.mean(sls_edge), np.mean(sls_normal)])       
    res.to_csv("RES_CIFAR_10_5CV(SLS_c0.5)_alphaimproved.csv", mode = 'a', float_format='%.4g')